##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cuantización entera postentrenamiento con activaciones int16

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/performance/post_training_integer_quant_16x8"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lite/performance/post_training_integer_quant_16x8.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lite/performance/post_training_integer_quant_16x8.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lite/performance/post_training_integer_quant_16x8.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

## Visión general

[TensorFlow Lite](https://www.tensorflow.org/lite/) admite ahora la conversión de las activaciones a valores enteros de 16 bits y de las ponderaciones a valores enteros de 8 bits durante la conversión del modelo de TensorFlow al formato flatbuffer de TensorFlow Lite. Nos referimos a este modo como el "modo de cuantización 16x8". Este modo puede mejorar significativamente la precisión del modelo cuantizado, cuando las activaciones son sensibles a la cuantización, al tiempo que consigue reducir casi 3-4 veces el tamaño del modelo. Además, este modelo cuantizado en su totalidad puede ser consumido por aceleradores de hardware de sólo enteros.

Algunos ejemplos de modelos que resultan útiles de este modo de la cuantización postentrenamiento incluyen:

- superresolución,
- procesamiento de señales de audio, como la cancelación de ruido y la formación de haces,
- eliminación de ruido de la imagen,
- Reconstrucción HDR a partir de una sola imagen

En este tutorial, usted entrena un modelo MNIST desde cero, revisa su precisión en TensorFlow y luego convierte el modelo en un flatbuffer de Tensorflow Lite usando este modo. Para el final, usted revisa la precisión del modelo convertido y lo compara con el modelo float32 original. Tenga en cuenta que este ejemplo demuestra el uso de este modo y no muestra beneficios sobre otras técnicas de cuantización disponibles en TensorFlow Lite.

## Generar un modelo MNIST

### Prepararación

In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib

Revise que el modo de cuantización 16x8 esté disponible 

In [ ]:
tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8

### Entrenar y exportar el modelo

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_data=(test_images, test_labels)
)

Para el ejemplo, usted ha entrenado el modelo durante una sola época, por lo que sólo se entrena con una precisión del ~96%.

### Convertir a un modelo TensorFlow Lite

Usando el [Convertidor](https://www.tensorflow.org/lite/models/convert) de TensorFlow Lite, ahora puede convertir el modelo entrenado en un modelo TensorFlow Lite.

Ahora, convierta el modelo utilizando `TFliteConverter` al formato predeterminado float32:

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Escríbalo en un archivo `.tflite`:

In [ ]:
tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
tflite_model_file = tflite_models_dir/"mnist_model.tflite"
tflite_model_file.write_bytes(tflite_model)

Para cuantizar el modelo en el modo de cuantización 16x8, confiqure primero el indicador `optimizations` para usar las optimizaciones predeterminadas. A continuación, especifique que el modo de cuantización 16x8 es la operación admitida requerida en la especificación del objetivo:

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]

Como en el caso de la cuantización postentrenamiento de int8, es posible producir un modelo cuantizado completamente entero ajustando las opciones del conversor `inference_input(output)_type` a tf.int16.

Configure los datos de calibración:

In [ ]:
mnist_train, _ = tf.keras.datasets.mnist.load_data()
images = tf.cast(mnist_train[0], tf.float32) / 255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]
converter.representative_dataset = representative_data_gen

Por último, convierta el modelo como de costumbre. Tenga en cuenta que, de forma predeterminada, el modelo convertido seguirá usando entradas y salidas de tipo float por comodidad de invocación.

In [ ]:
tflite_16x8_model = converter.convert()
tflite_model_16x8_file = tflite_models_dir/"mnist_model_quant_16x8.tflite"
tflite_model_16x8_file.write_bytes(tflite_16x8_model)

Observe cómo el archivo resultante, tiene aproximadamente `1/3` del tamaño.

In [ ]:
!ls -lh {tflite_models_dir}

## Ejecutar los modelos TensorFlow Lite

Ejecute el modelo TensorFlow Lite usando el intérprete TensorFlow Lite de Python.

### Cargar el modelo en los intérpretes

In [ ]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

In [ ]:
interpreter_16x8 = tf.lite.Interpreter(model_path=str(tflite_model_16x8_file))
interpreter_16x8.allocate_tensors()

### Probar los modelos en una imagen

In [ ]:
test_image = np.expand_dims(test_images[0], axis=0).astype(np.float32)

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

interpreter.set_tensor(input_index, test_image)
interpreter.invoke()
predictions = interpreter.get_tensor(output_index)

In [ ]:
import matplotlib.pylab as plt

plt.imshow(test_images[0])
template = "True:{true}, predicted:{predict}"
_ = plt.title(template.format(true= str(test_labels[0]),
                              predict=str(np.argmax(predictions[0]))))
plt.grid(False)

In [ ]:
test_image = np.expand_dims(test_images[0], axis=0).astype(np.float32)

input_index = interpreter_16x8.get_input_details()[0]["index"]
output_index = interpreter_16x8.get_output_details()[0]["index"]

interpreter_16x8.set_tensor(input_index, test_image)
interpreter_16x8.invoke()
predictions = interpreter_16x8.get_tensor(output_index)

In [ ]:
plt.imshow(test_images[0])
template = "True:{true}, predicted:{predict}"
_ = plt.title(template.format(true= str(test_labels[0]),
                              predict=str(np.argmax(predictions[0]))))
plt.grid(False)

### Evaluar los modelos

In [ ]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for test_image in test_images:
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  accurate_count = 0
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == test_labels[index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)

  return accuracy

In [ ]:
print(evaluate_model(interpreter))

Repita la evaluación en el modelo cuantizado de 16x8:

In [ ]:
# NOTE: This quantization mode is an experimental post-training mode,
# it does not have any optimized kernels implementations or
# specialized machine learning hardware accelerators. Therefore,
# it could be slower than the float interpreter.
print(evaluate_model(interpreter_16x8))

En este ejemplo, se ha cuantizado un modelo a 16x8 sin diferencia en la precisión, pero con el tamaño reducido 3 veces.
